# Importing Imdb Movie data
In this notebook we are retrieving basic data of movies and their popularity score - all published by imdb. In order to do so, we firstly download ```title.basics.tsv.gz``` and ```title.ratings.tsv.gz``` from ```https://datasets.imdbws.com/```. Secondly, we put the downloaded data in the folder ```data```. Thirdly, we load in the newly obtained data and merge it with our given movie data set. Lastly, we analyse how more or less movie data imdb provides compared to our given data.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#loading in imdb's movie data
movies_imdb = pd.read_csv('./data/title.basics.tsv/data.tsv', sep='\t')
movies_imdb.head()

In [ ]:
#loading in imdb's movie ratings
ratings_imdb = pd.read_csv('./data/title.ratings.tsv/data.tsv', sep='\t')
ratings_imdb.head()

In [ ]:
# TODO
# add rating to movie data set 

# merge imdb movie data with given movie data in order to have imdb movie id and ratings (only keep all movies in given movie data)

# compare how many movies are in imdb and how many are in the given movie data set
